In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open('input.txt', 'r') as f:
    text = f.read()


In [3]:
print(len(text))
print(text[:100])

217139351


April
April is the fourth month of the year in the Julian and Gregorian calendars, and comes betwe


In [4]:
chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)

In [5]:
len(chars)

94

In [6]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

In [69]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device', device)

device cuda


In [70]:
data = torch.tensor(encode(text), dtype=torch.long, device=device)

In [71]:
print(data.shape)

torch.Size([217139351])


In [72]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [73]:
block_size = 8
batch_size = 4
embed_dim = 32
learning_rate = 1e-3
max_iters = 10000
eval_interval = 1000
eval_iters = 1000

In [76]:
def get_batch(split):
    data = {
        'train': train_data,
        'val': val_data}[split]

    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)

    return x, y

In [84]:
class AttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        self.linear_v = nn.Linear(embed_dim, head_size)
        self.linear_k = nn.Linear(embed_dim, head_size)
        self.linear_q = nn.Linear(embed_dim, head_size)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inp):
        batch_dim, time_dim, channels = inp.shape
        
        values = self.linear_v(inp)
        keys = self.linear_k(inp)
        queries = self.linear_q(inp)

        W = keys @ queries.transpose(-2, -1) * channels**-0.5 # (batch_dim, time_dim, time_dim)
        W = W.masked_fill(self.tril[:time_dim, :time_dim] == 0, float('-inf'))
        W = F.softmax(W, dim=-1)

        out = W @ values

        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        self.mh_attention = MultiHeadAttention(4, embed_dim//4)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, inp, targets=None):
        batch_dim, time_dim = inp.shape
        
        token_emb = self.token_embedding(inp)
        pos_emb = self.pos_embedding(torch.arange(time_dim, device=device))

        x = token_emb + pos_emb
        x = self.mh_attention(x)
        x = self.lm_head(x)
        
        logits = x

        batch_dim, time_dim, channels = logits.shape

        if targets is None:
            loss = None
        else:
            logits = logits.view(batch_dim * time_dim, channels)
            targets = targets.view(batch_dim * time_dim)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, context, max_new_tokens):
        for i in range(max_new_tokens):
            context_cur = context[:, -block_size:]
            
            logits, loss = self(context_cur)

            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            new_tokens = torch.multinomial(probs, 1)

            context = torch.cat([context, new_tokens], dim=-1)

        return context

In [86]:
model = LanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

model = model.to(device)

In [87]:
@torch.no_grad()
def eval_loss(split):
    losses = torch.zeros(eval_iters)
    for i in range(eval_iters):
        xb, yb = get_batch(split)

        logits, loss = model(xb, yb)
        losses[i] = loss.item()
    
    return losses.mean()

In [88]:
for steps in range(max_iters):
    if steps % eval_interval == 0:
        print(f"step {steps:4d}: train loss: {eval_loss('train'):.8f}, val_loss: {eval_loss('val'):.8f}")
    
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step    0: train loss: 4.61199999, val_loss: 4.61380196
step 1000: train loss: 2.90992236, val_loss: 2.91970110
step 2000: train loss: 2.74776030, val_loss: 2.77228498
step 3000: train loss: 2.69041562, val_loss: 2.70657802
step 4000: train loss: 2.62865210, val_loss: 2.65481520
step 5000: train loss: 2.61753535, val_loss: 2.66378903
step 6000: train loss: 2.59393644, val_loss: 2.63510609
step 7000: train loss: 2.57061172, val_loss: 2.61380076
step 8000: train loss: 2.54449272, val_loss: 2.58242464
step 9000: train loss: 2.55085301, val_loss: 2.57622766


In [89]:
eval_loss('val')

tensor(2.5599)

In [90]:
context = torch.tensor([[0]], device=device)
context = model.generate(context, 200)
context = context.view(-1).tolist()
print(decode(context))


Intilall, Ree of Aburg Epasty Cu, rado of Wof verd loulacovapony of dinne of thire keedd eand Miniof ofn thes. Pe Maths than of Fra sco
Blet dot thortt be.
Norony heotenvithans, na Alebude a 19813)
19


In [66]:
print(torch.cuda.is_available())

True
